In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.spatial.distance import squareform

In [ ]:
data_path = "./data/data_with_human_TE_cellline_all_plain.csv"
df = pd.read_csv(data_path, delimiter="\t")
df.head()

In [ ]:
#number of unique cell lines -- number of columns that have "bio_source" in the name

print("Number of columns: ", len(df.columns))
print("Number of rows: ", len(df))

na_rows = df[df.isna().any(axis=1)]
print("Number of rows that have NA: ", len(na_rows))

bio_source_cols = [col for col in df.columns if 'bio_source' in col]
print(f"Number of unique human cell lines: {len(bio_source_cols)}")

In [ ]:
#fill in the null values
df = df.dropna()
na_rows = df[df.isna().any(axis=1)]
print("Number of rows that have NA: ", len(na_rows))
print("Number of rows: ", len(df))
df.head()


In [ ]:
bio_source_cols = [col for col in df.columns if 'bio_source' in col]
bio_source_df = df[bio_source_cols]
bio_source_df.columns = bio_source_df.columns.str.replace('bio_source_', '')

In [ ]:
bio_source_df.head()

In [ ]:
spearman_corr = bio_source_df.corr(method='spearman')
print("Dimensions of spearman correlation matrix: ", spearman_corr.shape)

In [ ]:
distance_matrix = 1 - spearman_corr
condensed_dist = squareform(distance_matrix.values)
linkage_matrix = linkage(condensed_dist, method='average')

ordered_indices = leaves_list(linkage_matrix)
ordered_corr = spearman_corr.iloc[ordered_indices, ordered_indices]

mask = np.triu(np.ones_like(ordered_corr, dtype=bool))

plt.figure(figsize=(20, 18))
sns.heatmap(
    ordered_corr, 
    mask=mask, 
    cmap='coolwarm', 
    annot=False, 
    fmt=".2f",
    linewidths=0.5, 
    square=True, 
    cbar_kws={"shrink": 0.5}
)

plt.xticks(rotation=90, fontsize=8)
plt.yticks(fontsize=8)
plt.title("Spearman Correlation (Hierarchically Clustered)", fontsize=16)
plt.tight_layout()
plt.savefig('spearman_correlation_heatmap.png')
plt.show()


In [ ]:
# Import necessary libraries
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Use LinReg model to plot predictions againt observed Mean TE and calculate R^2, Pearson, and Spearman

In [ ]:
# train linear regression using solely the bio_source
# Filter columns for inputs (columns 3:6) and outputs (columns starting with 'bio_source_')
input_columns = df.iloc[:, 3:7]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_columns, bio_source_df, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)
print(y_pred.shape)

#average the predictions and observations by row
y_pred_avg = np.mean(y_pred, axis=1)
y_test_avg = np.mean(y_test, axis=1)
print(y_pred_avg.shape)

# Evaluate the model
mse = mean_squared_error(y_test_avg, y_pred_avg)
print(f"Mean Squared Error: {mse}")

# Create 2D histogram
plt.figure(figsize=(6, 6))
hb = plt.hexbin(y_pred_avg, y_test_avg, gridsize=100, cmap='hot', mincnt=1)

# Add colorbar
cb = plt.colorbar(hb)
cb.set_label('Density')

# Labels and title
plt.xlabel('LinReg prediction (human)')
plt.ylabel('Observed Mean TE (human)')
plt.title('Density Scatter Plot')

# Optional: Set axis limits similar to your image
plt.xlim(-1, 2)
plt.ylim(-2, 3)

plt.show()

### Metrics code

In [17]:
from scipy.stats import spearmanr, pearsonr

print(r2_score(y_test_avg, y_pred_avg))
print(spearmanr(y_test_avg, y_pred_avg).correlation)
print(pearsonr(y_test_avg, y_pred_avg).statistic)

0.2547182146077034
0.5923102212218953
0.5048773638065486


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
import torch.nn as nn

# Function to split a sequence into 3-mers
def tokenize_sequence(sequence, k=3):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

# Extract the tx_sequence column
sequences = df['tx_sequence']

# # Tokenize each sequence into 3-mers
tokenized_sequences = sequences.apply(tokenize_sequence)

# Create a vocabulary of unique 3-mers and map them to indices
unique_kmers = set(kmer for seq in tokenized_sequences for kmer in seq)
kmer_to_index = {kmer: idx for idx, kmer in enumerate(unique_kmers)}

# Define the embedding layer
vocab_size = len(kmer_to_index)
embedding_dim = 50  # You can adjust the embedding dimension
embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# Convert each sequence of 3-mers into embeddings
def sequence_to_embeddings(sequence):
    encoded_sequence = [kmer_to_index[kmer] for kmer in sequence]
    encoded_sequence_tensor = torch.tensor(encoded_sequence, dtype=torch.long)
    return embedding_layer(encoded_sequence_tensor).detach().numpy()

# Map each sequence to its embeddings
bio_source_df['embeddings'] = tokenized_sequences.apply(sequence_to_embeddings)

# Example: Access embeddings for the first sequence
print(bio_source_df['embeddings'].iloc[0])

# Uses too much memory - Flatten the list of 3-mers and encode them as integers
# all_kmers = [kmer for seq in tokenized_sequences for kmer in seq]
# label_encoder = LabelEncoder()
# encoded_kmers = label_encoder.fit_transform(all_kmers)

In [ ]:
# create a mapping from kmer to its embedding 
# graph embeddings of a given k-mer to visualize similarity
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

# Create a mapping between k-mers and their embeddings
kmer_to_embedding = {kmer: embedding_layer(torch.tensor([idx], dtype=torch.long)).detach().numpy().flatten()
                     for kmer, idx in kmer_to_index.items()}

# Extract k-mers and their embeddings
kmers = list(kmer_to_embedding.keys())
embeddings = np.array(list(kmer_to_embedding.values()))

In [ ]:
print(len(embeddings))

In [ ]:
# Reduce dimensionality of embeddings for visualization
# You can use PCA or t-SNE
def reduce_embeddings(embeddings, method='pca', n_components=2):
    if method == 'pca':
        reducer = PCA(n_components=n_components)
    elif method == 'tsne':
        reducer = TSNE(n_components=n_components, random_state=42)
    else:
        raise ValueError("Invalid method. Use 'pca' or 'tsne'.")
    return reducer.fit_transform(embeddings)

# Reduce to 2D for visualization
reduced_embeddings = reduce_embeddings(embeddings[:30], method='tsne', n_components=2)

# Plot the reduced embeddings
plt.figure(figsize=(10, 8))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], alpha=0.7, s=50)

In [ ]:
# # Create a mapping from 3-mers to embeddings
# vocab_size = len(label_encoder.classes_)
# embedding_dim = 50  # You can adjust the embedding dimension
# embedding_layer = nn.Embedding(vocab_size, embedding_dim)

# # Convert each sequence of 3-mers into embeddings
# def sequence_to_embeddings(sequence):
#     encoded_sequence = label_encoder.transform(sequence)
#     encoded_sequence_tensor = torch.tensor(encoded_sequence, dtype=torch.long)
#     return embedding_layer(encoded_sequence_tensor).detach().numpy()

# # Map each sequence to its embeddings
# bio_source_df['embeddings'] = tokenized_sequences.apply(sequence_to_embeddings)

# # Example: Access embeddings for the first sequence
# print(bio_source_df['embeddings'].iloc[0])